# Check how it is working

In [ ]:
> high level detals--> individual low level
functions:
1) how opening and closing of the video capture works
2) check the _generate_report function

## import all the libraries

In [2]:
import cv2
import numpy as np
import xgboost as xgb
import mediapipe as mp
from dataclasses import dataclass
from enum import Enum
from typing import List, Tuple
import pickle
import matplotlib.pyplot as plt
from src.feature_extraction import FeatureExtractor
from src.inference import FraudDetectionInference
from app import RiskLevel
from app import FrameAnalysis
from app import ExamReport
from app import VideoProcessor

class RiskLevel(Enum):
    MINIMAL = "MINIMAL"
    LOW = "LOW"
    MODERATE = "MODERATE"
    HIGH = "HIGH"
    CRITICAL = "CRITICAL"
    
@dataclass
class FrameAnalysis:
    frame_id: int
    timestamp: float
    exam_id: str
    xgboost_probability: float
    frame_image: np.ndarray = None

@dataclass
class ExamReport:
    exam_id: str
    peak_score: float
    risk_level: RiskLevel
    peak_frame_id: int
    peak_frame_timestamp: float
    peak_frame_image: np.ndarray
    total_frames: int
    recommendation: str

In [3]:
# Define Global params
video_path = r"E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\Example Images\input.mp4"
exam_id = "exam_001"
fps_sample=1
max_frames=300

In [4]:
vp=VideoProcessor(xgb_model_path=r"E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\models\xgboost_fraud_detection_model.pkl")
f=FeatureExtractor()
cap=cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise ValueError(f"Cannot open video file: {video_path}")
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print("Total number of frames:", total_frames)
fps = cap.get(cv2.CAP_PROP_FPS)

if fps <= 0:
    fps = 25 # Default FPS if unable to get from video

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"Total frames: {total_frames}")
print(f"FPS: {fps}")
print(f"Resolution: {width}x{height}")

frame_analysis=[]
frame_count=0
processed_count=0

try:
    inference=FraudDetectionInference()
    print("✓ Model loaded successfully.\n")
except FileNotFoundError as e:
    raise RuntimeError("Model file not found. Please ensure the model is available.") from e

print("Starting video processing...\n")

try:
    while cap.isOpened():
        ret,frame=cap.read()
        print("Hey bro, I am coming here")
        if not ret:
            print("End of video stream or cannot read the frame.")
            break
        frame_count+=1
        # Frame Sampling
        print("Krishna is great")
        if fps_sample > 1 and frame_count % fps_sample !=0:
            continue
        print("Jai Shri Ram")
        if max_frames is not None and processed_count >= max_frames:
            break
        print("Hare Ram")
        # Feature Extraction
        features = f.extract_features(frame)
        
        if features is None:
            print(f"Frame {frame_count}: No features detected.")
            continue

        # Fraud Prediction
        result = inference.predict(features)
        xgb_prob = float(result.get("fraud_probability", 0.0))
        xgb_prob=result['fraud_probability']
        print("Hare Krishna, krishna is here, why fear")
        # Accurate timestamp 
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC)/1000.0  # in seconds
        print(f"Prediction Score: {result['fraud_probability']:.4f}")
        processed_count=processed_count+1

        frame_analysis.append(FrameAnalysis(
            frame_id=frame_count,
            timestamp=timestamp,
            exam_id=exam_id,
            xgboost_probability=xgb_prob,
            frame_image=None
        ))

        # Display progress
        if processed_count>0 and processed_count % 30 ==0:
            print(f"Processed {processed_count} frames...")

finally:
    print("Releasing video capture resources.")
    cap.release()

print("\n Finished Processing all frames.")
print(f"Total Processed Frames: {processed_count}\n")
# report=vp._generate_report(frame_analysis,exam_id)

✓ Loaded XGBoost model from E:\Projects in ML\FRAUD DETECTION SYSTEM FOR THE ONLINE PROCTORED EXAMS\models\xgboost_fraud_detection_model.pkl
Total number of frames: 37
Total frames: 37
FPS: 15.205271160669032
Resolution: 1280x720
Model and preprocessor loaded successfully from models
✓ Model loaded successfully.

Starting video processing...

Hey bro, I am coming here
Krishna is great
Jai Shri Ram
Hare Ram

0: 384x640 1 person, 845.5ms
Speed: 67.1ms preprocess, 845.5ms inference, 54.4ms postprocess per image at shape (1, 3, 384, 640)
Hare Krishna, krishna is here, why fear
Prediction Score: 0.9954
Hey bro, I am coming here
Krishna is great
Jai Shri Ram
Hare Ram

0: 384x640 1 person, 76.6ms
Speed: 2.5ms preprocess, 76.6ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Hare Krishna, krishna is here, why fear
Prediction Score: 0.9957
Hey bro, I am coming here
Krishna is great
Jai Shri Ram
Hare Ram

0: 384x640 1 person, 72.6ms
Speed: 1.8ms preprocess, 72.6ms inference, 1.

AttributeError: 'list' object has no attribute 'classification'

In [5]:
frame_analysis

[FrameAnalysis(frame_id=1, timestamp=0.0, exam_id='exam_001', xgboost_probability=0.9954321384429932, frame_image=None),
 FrameAnalysis(frame_id=2, timestamp=0.08006666666666667, exam_id='exam_001', xgboost_probability=0.9957340359687805, frame_image=None),
 FrameAnalysis(frame_id=3, timestamp=0.14386666666666667, exam_id='exam_001', xgboost_probability=0.9954321384429932, frame_image=None),
 FrameAnalysis(frame_id=4, timestamp=0.2083, exam_id='exam_001', xgboost_probability=0.9957340359687805, frame_image=None),
 FrameAnalysis(frame_id=5, timestamp=0.2722, exam_id='exam_001', xgboost_probability=0.9954321384429932, frame_image=None)]

In [6]:
report=vp._generate_report(frame_analysis,exam_id)

In [7]:
report

ExamReport(exam_id='exam_001', peak_score=0.9957, risk_level=<RiskLevel.CRITICAL: 'CRITICAL'>, peak_frame_id=2, peak_frame_timestamp=0.08, peak_frame_image=None, total_frames=5, recommendation='🚨 CRITICAL FRAUD RISK: Exam shows severe suspicious indicators. RECOMMEND: Immediately investigate and consider invalidating exam.')